#Worcester Crash Reports

This project is based on the vehicular crash reports from the Worcester County. Using the mapping tool from the MASSDOT [crash portal](http://services.massdot.state.ma.us/crashportal/CrashMapPage.aspx?Mode=Mapping). I tried to grab all the data I could get, but since you could only get 8000 results at once, I had to do it multiple times to get the entire city of Worcester's data.

In [252]:
import pandas as pd #For Data Cleaning and Manipulation
import matplotlib as plt #Data analysis by plotting
import glob #Used as a way to open all csv files that were collected
import datetime #used to clean and better express the Date and Time features
#import time

Joining all csv files into one dataframe
Since I had to download multiple csv files, I needed to combine all of them into one data. I could of open them one by one and assigning them to a dataframe, but I decided to look around for a solution into reading all the csv files and then concatinating it into a bigger csv file. I then decided to remove all the duplicates if there was any, so that we  wouldn't have repeated data.

In [253]:
worcester_crashreports = glob.glob("*.csv") 
df = pd.concat((pd.read_csv(f, header = 0, low_memory=False) for f in worcester_crashreports), sort= False)
df_worcrash = df.drop_duplicates()
#df_worcrash.to_csv("Worces.csv")

In [254]:
df_worcrash.head(5) #Just looking at what our data looks like.

,Crash Number,Crash Date,Crash Time,City/Town,Crash Severity,Roadway,Near Intersection Roadway,Landmark,Police Agency,Unnamed: 0
0,4051393,3/24/2015,11:47 AM,WORCESTER,Non-fatal injury,SALISBURY STREET / WESTWOOD DRIVE,NaN,NaN,Local police,NaN
1,4051388,3/24/2015,2:25 PM,WORCESTER,Property damage only (none injured),CAMELOT DRIVE,NaN,NaN,Local police,NaN
2,4054884,2/26/2015,11:10 AM,WORCESTER,Property damage only (none injured),CHESTER STREET,NaN,NaN,Local police,NaN
3,2790347,11/1/2011,8:25 PM,WORCESTER,Property damage only (none injured),Rte 190 S,NaN,NaN,State police,NaN
4,4054867,2/26/2015,8:29 PM,WORCESTER,Property damage only (none injured),MOUNTAIN STREET WEST / BROOKS STREET,NaN,NaN,Local police,NaN


##Data Cleaning

Renaming the Columns for easier syntax

In [257]:
df_worcrash = df.rename(columns = {'Crash Date':'CrashDate','Crash Time':'CrashTime', 'Crash Severity':'CrashSev', 'Police Agency':'PoliceAgency'})

In [258]:
df_worcrash.head(9)

,Crash Number,CrashDate,CrashTime,City/Town,CrashSev,Roadway,Near Intersection Roadway,Landmark,PoliceAgency,Unnamed: 0
0,4051393,3/24/2015,11:47 AM,WORCESTER,Non-fatal injury,SALISBURY STREET / WESTWOOD DRIVE,NaN,NaN,Local police,NaN
1,4051388,3/24/2015,2:25 PM,WORCESTER,Property damage only (none injured),CAMELOT DRIVE,NaN,NaN,Local police,NaN
2,4054884,2/26/2015,11:10 AM,WORCESTER,Property damage only (none injured),CHESTER STREET,NaN,NaN,Local police,NaN
3,2790347,11/1/2011,8:25 PM,WORCESTER,Property damage only (none injured),Rte 190 S,NaN,NaN,State police,NaN
4,4054867,2/26/2015,8:29 PM,WORCESTER,Property damage only (none injured),MOUNTAIN STREET WEST / BROOKS STREET,NaN,NaN,Local police,NaN
5,4055722,2/19/2015,3:58 PM,WORCESTER,Property damage only (none injured),CHESTER STREET,NaN,NaN,Local police,NaN
6,4055774,5/17/2015,1:00 PM,WORCESTER,Non-fatal injury,WEST BOYLSTON STREET / WEST BOYLSTON DRIVE,NaN,NaN,Local police,NaN
7,2790266,11/1/2011,10:00 PM,WORCESTER,Non-fatal injury,Rte 190 N,NaN,NaN,State police,NaN
8,2791027,7/1/2010,00:98 AM,WORCESTER,Not Reported,BLUEBELL ROAD,NaN,NaN,Local police,NaN


Checking whether I grabbed some data from other towns

In [260]:
df_worcrash['City/Town'].unique()

array(['WORCESTER', 'WEST BOYLSTON', 'HOLDEN', 'GRAFTON', nan,
       'SHREWSBURY', 'HUDSON', 'LEICESTER', 'MILLBURY', 'AUBURN',
       'LEOMINSTER'], dtype=object)

In [261]:
df_worcrash = df_worcrash[df_worcrash['City/Town'] == 'WORCESTER'] #Only selecting Worcester from the files

In [262]:
df_worcrash.head(5) #This is what the data looks like now.

,Crash Number,CrashDate,CrashTime,City/Town,CrashSev,Roadway,Near Intersection Roadway,Landmark,PoliceAgency,Unnamed: 0
0,4051393,3/24/2015,11:47 AM,WORCESTER,Non-fatal injury,SALISBURY STREET / WESTWOOD DRIVE,NaN,NaN,Local police,NaN
1,4051388,3/24/2015,2:25 PM,WORCESTER,Property damage only (none injured),CAMELOT DRIVE,NaN,NaN,Local police,NaN
2,4054884,2/26/2015,11:10 AM,WORCESTER,Property damage only (none injured),CHESTER STREET,NaN,NaN,Local police,NaN
3,2790347,11/1/2011,8:25 PM,WORCESTER,Property damage only (none injured),Rte 190 S,NaN,NaN,State police,NaN
4,4054867,2/26/2015,8:29 PM,WORCESTER,Property damage only (none injured),MOUNTAIN STREET WEST / BROOKS STREET,NaN,NaN,Local police,NaN


Changing our data set and only selecting specific columns. Seems like were not gonna need the City/Town, Landmark and Near Intersection Roadway features since its mostly not available.

In [264]:
df_worcrash = df_worcrash[[ 'CrashDate','CrashTime', 'CrashSev','PoliceAgency','Roadway']]

In [265]:
df_worcrash.head(10)

,CrashDate,CrashTime,CrashSev,PoliceAgency,Roadway
0,3/24/2015,11:47 AM,Non-fatal injury,Local police,SALISBURY STREET / WESTWOOD DRIVE
1,3/24/2015,2:25 PM,Property damage only (none injured),Local police,CAMELOT DRIVE
2,2/26/2015,11:10 AM,Property damage only (none injured),Local police,CHESTER STREET
3,11/1/2011,8:25 PM,Property damage only (none injured),State police,Rte 190 S
4,2/26/2015,8:29 PM,Property damage only (none injured),Local police,MOUNTAIN STREET WEST / BROOKS STREET
5,2/19/2015,3:58 PM,Property damage only (none injured),Local police,CHESTER STREET
6,5/17/2015,1:00 PM,Non-fatal injury,Local police,WEST BOYLSTON STREET / WEST BOYLSTON DRIVE
7,11/1/2011,10:00 PM,Non-fatal injury,State police,Rte 190 N
8,7/1/2010,00:98 AM,Not Reported,Local police,BLUEBELL ROAD
9,5/19/2010,10:30 AM,Property damage only (none injured),Local police,WEST BOYLSTON STREET


Looking at the data, I noticed some of them have a time '00:98' which doesn't not make sense, so I turned them to '1:30'

In [267]:
df_worcrash.loc[df_worcrash['CrashTime'] == '00:98 AM', 'CrashTime' ] = '01:30 AM'

Combining the CrashDate and CrashTime to make a datetime property

In [269]:
df_worcrash['CrashDT'] = df_worcrash['CrashDate'] + ' ' + df_worcrash['CrashTime']
df_worcrash.head(5)

,CrashDate,CrashTime,CrashSev,PoliceAgency,Roadway,CrashDT
0,3/24/2015,11:47 AM,Non-fatal injury,Local police,SALISBURY STREET / WESTWOOD DRIVE,3/24/2015 11:47 AM
1,3/24/2015,2:25 PM,Property damage only (none injured),Local police,CAMELOT DRIVE,3/24/2015 2:25 PM
2,2/26/2015,11:10 AM,Property damage only (none injured),Local police,CHESTER STREET,2/26/2015 11:10 AM
3,11/1/2011,8:25 PM,Property damage only (none injured),State police,Rte 190 S,11/1/2011 8:25 PM
4,2/26/2015,8:29 PM,Property damage only (none injured),Local police,MOUNTAIN STREET WEST / BROOKS STREET,2/26/2015 8:29 PM


In [270]:
df_worcrash['CrashDT'] = pd.to_datetime(df_worcrash['CrashDT'], format = '%m/%d/%Y %I:%M %p')

In [271]:
df_worcrash = df_worcrash[['CrashDT', 'CrashSev', 'PoliceAgency', 'Roadway']] #Just another clean up. This looks much better now.
df_worcrash.head(5)

,CrashDT,CrashSev,PoliceAgency,Roadway
0,2015-03-24 11:47:00,Non-fatal injury,Local police,SALISBURY STREET / WESTWOOD DRIVE
1,2015-03-24 14:25:00,Property damage only (none injured),Local police,CAMELOT DRIVE
2,2015-02-26 11:10:00,Property damage only (none injured),Local police,CHESTER STREET
3,2011-11-01 20:25:00,Property damage only (none injured),State police,Rte 190 S
4,2015-02-26 20:29:00,Property damage only (none injured),Local police,MOUNTAIN STREET WEST / BROOKS STREET
